# The improper dihedral for benzene does not get written by ParmEd

This is a problem if someone wants to parameterize a system with SMIRNOFF99Frosst, and use ParmEd to save a `.frcmod` file so that AmberTools (e.g., `tleap`) can be used downstream to add solvent or otherwise manipulate the structure.

In [1]:
import numpy as np
import simtk

This is using the 0.3.0 development branch of the toolkit.

In [2]:
import openforcefield

In [3]:
openforcefield.__version__

'0.2.2+8.gd3c1bcf'

The starting point is a standard `.mol2` file with Tripos atom names.

In [4]:
with open("../data/benzene.tripos.mol2") as file:
    benzenze = file.read()

## AMBER workflow to create `.frcmod` file for benzene (for example)

Let's create an AMBER `frcmod` for reference:


```bash
antechamber -fi mol2 -i benzene.tripos.mol2 -fo mol2 -o benzene.gaff.mol2
tleap -f tleap.gaff.in
parmchk2 -i benzene.gaff.mol2 -f mol2 -o benzene.gaff.frcmod -s 2 -a Y
```

where `tleap.gaff.in` is:
```
source leaprc.gaff2
mol = loadmol2 benzene.gaff.mol2
saveamberparm mol benzene.gaff.prmtop benzene.gaff.inpcrd
```

The contents of the GAFF2 `frcmod` file is:

```
Remark line goes here
MASS
ca 12.010        0.360
ha 1.008         0.135

BOND
ca-ca  378.57   1.398
ca-ha  395.72   1.086

ANGLE
ca-ca-ca   68.767     120.020
ca-ca-ha   48.680     119.880

DIHE
ca-ca-ca-ca   4   14.500       180.000           2.000
ca-ca-ca-ha   4   14.500       180.000           2.000
ha-ca-ca-ha   4   14.500       180.000           2.000

IMPROPER
ca-ca-ca-ha         1.1          180.0         2.0          Using general improper torsional angle  X- X-ca-ha, penalty score=  6.0)

NONBON
  ca          1.8606  0.0988
  ha          1.4735  0.0161

```

## `openforcefield` toolkit workflow

Tripols MOL2 → OEMols → OpenMM system → MOL2, PRMTOP, FRCMOD (via ParmED)

In [5]:
from openforcefield.typing.engines.smirnoff import ForceField
from openforcefield.topology import Molecule, Topology

In [6]:
import parmed as pmd

### 1. Read in `.mol2`

In [7]:
mol = openforcefield.topology.Molecule.from_file("../data/benzene.tripos.mol2")

In [8]:
mol

Molecule with name 'LIG' and SMILES '[H]c1c(c(c(c(c1[H])[H])[H])[H])[H]'

### 2. Create topology

In [9]:
topology = openforcefield.topology.Topology.from_molecules([mol])

In [10]:
print(f"I see {len([i for i in topology.topology_atoms])} atoms.")


print(f"I see {len([i for i in topology.topology_bonds])} bonds.")
print(f"I see {topology.n_angles} angles.")
print(f"I see {topology.n_propers} dihedrals.")
print(f"I see {topology.n_impropers} impropers.")

I see 12 atoms.
I see 12 bonds.
I see 18 angles.
I see 24 dihedrals.
I see 36 impropers.


### 3. Parameterize

In [11]:
ff = ForceField("smirnoff99Frosst.offxml")

In [12]:
system = ff.create_openmm_system(topology)

In [13]:
print(f"I see {system.getNumParticles()} atoms.")

for force in system.getForces():
    if isinstance(force, simtk.openmm.openmm.HarmonicBondForce):
        print(f"I see {force.getNumBonds()} bonds.")
    if isinstance(force, simtk.openmm.openmm.HarmonicAngleForce):
        print(f"I see {force.getNumAngles()} angles.")
    if isinstance(force, simtk.openmm.openmm.PeriodicTorsionForce):
        print(f"I see {force.getNumTorsions()} mixed proper and improper dihedrals.")

I see 12 atoms.
I see 42 mixed proper and improper dihedrals.
I see 18 angles.
I see 12 bonds.


### 4. Bring into ParmEd

In [35]:
parmed_structure = pmd.openmm.load_topology(topology.to_openmm(), system)

In [36]:
print(f"I see {len(parmed_structure.atoms)} atoms.")


print(f"I see {len(parmed_structure.bonds)} bonds.")
print(f"I see {len(parmed_structure.angles)} angles.")
print(f"I see {len(parmed_structure.dihedrals)} mixed proper and improper dihedrals.")

I see 12 atoms.
I see 12 bonds.
I see 18 angles.
I see 42 mixed proper and improper dihedrals.


In ParmEd, I think propers and impropers are all in the same list. Only CHARMM-style impropers are included in `Structure.impropers`. See this comment: https://github.com/ParmEd/ParmEd/issues/990#issuecomment-397844139

We can directly query the distinct propers and impropers, though.

In [37]:
parmed_structure.dihedral_types

TrackedList([
	<DihedralType; phi_k=0.367, per=2, phase=180.000,  scee=1.000, scnb=1.000>
	<DihedralType; phi_k=3.625, per=2, phase=180.000,  scee=1.000, scnb=1.000>
])

**Potential bug 1. These dihedral scaling factors are wrong / not the ones intended.**

I'm almost positive these should be `scee=1.2` (i.e., 1.0/1.2 = 0.83333...) and `scnb=2.0` (i.e., 1.0/2.0 = 0.5) for the 1-4 interactions.

In [38]:
parmed_structure.improper_types

TrackedList([
])

Note the complete lack of impropers here.

In [39]:
parmed_structure.save("../data/benzene.smirnoff.prmtop", overwrite=True)

**Potential bug 2. We can't yet write a `.frcmod` because the atoms do not have names.**  There probably also should have been an error raised for writing the `.prmtop`

In [42]:
for index, atom in enumerate(parmed_structure.atoms):
    print(index, atom.name, atom.type)

0  C1
1  C1
2  C1
3  C1
4  C1
5  C1
6  H1
7  H1
8  H1
9  H1
10  H1
11  H1


**Potential bug 3. We cannot write out a `.frcmod`** This error looks new to me. Not sure yet how to debug.

In [44]:
for index, atom in enumerate(parmed_structure.atoms):
    atom.name = index
    
parm_set = pmd.amber.AmberParameterSet.from_structure(parmed_structure)
parm_set.write("../data/benzene.smirnoff.frcmod")

ParameterError: Unequal dihedral types defined bewteen C1, C1, C1, and H1

**Potential bug 4.** Even if a `.frcmod` could be written, the `IMPROPER` section would be empty, which is wrong. I'm not sure if (a) they get written to the `DIHEDRAL` section and get silently ignored by AMBER (at best undefined behavior, as far as I can tell) or (b) they are just not written at all. This may be a bug upstream of ParmEd, as it looks like the OpenMM `System` also doesn't have the correct number of torsions. I'm not 100% sure I understand what is going on here.

In [46]:
!conda list

# packages in environment at /home/davids4/data/anaconda3/envs/off-dev2:
#
# Name                    Version                   Build  Channel
ambermini                 16.16.0                       7    omnia
atomicwrites              1.3.0                    py37_1  
attrs                     19.1.0                   py37_1  
backcall                  0.1.0                    py37_0  
bleach                    3.1.0                    py37_0  
boost                     1.68.0          py37h8619c78_1001    conda-forge
boost-cpp                 1.68.0            h11c811c_1000    conda-forge
bson                      0.5.8                      py_0    conda-forge
bzip2                     1.0.6             h14c3975_1002    conda-forge
ca-certificates           2019.3.9             hecc5488_0    conda-forge
cairo                     1.16.0            ha4e643d_1000    conda-forge
certifi                   2019.3.9                 py37_0  
dbus                      1.13.6               he37